In [15]:
!pip install -q mcp langchain-mcp-adapters

In [16]:
import sys
import asyncio
from mcp import ClientSession
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

async def run_mcp(chat_history=None):
    if chat_history is None:
        chat_history = {"messages": []}

    math_params = StdioServerParameters(
        command=sys.executable,
        args=["C:\\Users\\dell\\Documents\\CODES\\MAT496\\MCP\\servers\\math_server.py"],
    )
    fetch_params = StdioServerParameters(
        command=sys.executable,
        args=["-m", "mcp_server_fetch"],
    )

    async with stdio_client(math_params, errlog=None) as (math_read, math_write):
        async with stdio_client(fetch_params, errlog=None) as (fetch_read, fetch_write):
            async with ClientSession(math_read, math_write) as math_session:
                async with ClientSession(fetch_read, fetch_write) as fetch_session:
                    await math_session.initialize()
                    await fetch_session.initialize()
                    math_tools = await load_mcp_tools(math_session)
                    fetch_tools = await load_mcp_tools(fetch_session)
                    tools = math_tools + fetch_tools
                    agent = create_react_agent("groq:openai/gpt-oss-120b", tools)
                    result = await agent.ainvoke(chat_history)
                    return result

ModuleNotFoundError: No module named 'langgraph'

In [ ]:
chat_history = {"messages": [{"role": "user", "content": "hi?"}]}

chat_history = await run_mcp(chat_history)
chat_history

In [ ]:
new_message = "Go the the site https://krajit.github.io/ and find Ajit's email address"

chat_history["messages"].append({"role": "user", "content": new_message})
chat_history = await run_mcp(chat_history)
chat_history